In [1]:
import time
import numpy as np
import torch as torch
#import tf_cnnvis
import logging
import importlib
import sys
import os
import torch.nn as nn

FREQ_PRINT = 200 # print frequency image tensorboard [20]
FREQ_EV = 1
PATIENCE = 10

In [2]:
from utils.adapt_data import batch_fill
from utils.evaluations import save_results, heatmap
from utils.constants import IMAGES_DATASETS

from torchsummary import summary



/home/austrotaxus/anaconda3/envs/marsanomalies/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
learning_rate = 0.0002
batch_size = 32
latent_dim = 100

dataset = "cifar10"
label =1 
device = 'cuda'



In [4]:
def display_parameters(batch_size, starting_lr, ema_decay, degree, label,
                       allow_zz, score_method, do_spectral_norm):
    """See parameters
    """
    print('Batch size: ', batch_size)
    print('Starting learning rate: ', starting_lr)
    print('EMA Decay: ', ema_decay)
    print('Degree for L norms: ', degree)
    print('Anomalous label: ', label)
    print('Score method: ', score_method)
    print('Discriminator zz enabled: ', allow_zz)
    print('Spectral Norm enabled: ', do_spectral_norm)

def display_progression_epoch(j, id_max):
    """See epoch progression
    """
    batch_progression = int((j / id_max) * 100)
    sys.stdout.write(str(batch_progression) + ' % epoch' + chr(13))
    _ = sys.stdout.flush

In [5]:
def create_logdir(dataset, label, rd,
                  allow_zz, score_method, do_spectral_norm):
    """ Directory to save training logs, weights, biases, etc."""
    model = 'alad_sn{}_dzz{}'.format(do_spectral_norm, allow_zz)
    return "train_logs/{}/{}/dzzenabled{}/{}/label{}/" \
           "rd{}".format(dataset, model, allow_zz,
            score_method, label, rd)

In [6]:
"""
CIFAR10 ALAD architecture.
Generator (decoder), encoder and discriminator.
"""
import torch as torch
#from utils import sn

#init_kernel = tf.random_normal_initializer(mean=0.0, stddev=0.01)

class Encoder_Network(nn.Module):
    def __init__(self, is_training=False, getter=None, reuse=False,
            do_spectral_norm=True):
        """ Encoder architecture in tensorflow
        Maps the data into the latent space
        Args:
            x_inp (tensor): input data for the encoder.
            is_training (bool): for batch norms and dropouts
            getter: for exponential moving average during inference
            reuse (bool): sharing variables or not
        Returns:
        net (tensor): last activation layer of the encoder
        """
        super(Encoder_Network, self).__init__()
        layers = []


        layers.append(torch.nn.Conv2d(3,128,kernel_size = 4,stride = 2, padding = 1))
        layers.append(torch.nn.BatchNorm2d(128))
        layers.append(torch.nn.LeakyReLU(negative_slope=0.2))

        layers.append(torch.nn.Conv2d(128,256,kernel_size = 4,stride = 2, padding = 1))
        layers.append(torch.nn.BatchNorm2d(256))
        layers.append(torch.nn.LeakyReLU(negative_slope=0.2))

        layers.append(torch.nn.Conv2d(256,512,kernel_size = 4,stride = 2, padding = 1))
        layers.append(torch.nn.BatchNorm2d(512))
        layers.append(torch.nn.LeakyReLU(negative_slope=0.2))

        
        layers.append(torch.nn.Conv2d(512,latent_dim,kernel_size =4,stride = 1, padding = 0))
        #layers.append(torch.nn.BatchNorm2d(batch_size)
        #layers.append(torch.nn.LeakyRelu(negative_slope=0.2))    
        self.enc = nn.Sequential(*layers).to(device)
    
    def forward(self,x):
        return self.enc(x.to(device))


class Decoder_Network(nn.Module):
    def __init__(self,is_training=False, getter=None, reuse=False,
            do_spectral_norm=True):
        
        norm = lambda x : nn.utils.spectral_norm(x) if do_spectral_norm else lambda x: x
        super(Decoder_Network, self).__init__()
                      
        layers = []

            
 #       x_inp = x_inp.view(-1, 32, 32, 3)

        layers.append(norm(
            nn.ConvTranspose2d(latent_dim,512,kernel_size = 4,stride = 1, padding = 0)
            ))
    
        layers.append(torch.nn.BatchNorm2d(512))
        layers.append(nn.ReLU())

        layers.append(norm(nn.ConvTranspose2d(512,256,kernel_size =  4,stride = 2, padding = 1)))
        layers.append(torch.nn.BatchNorm2d(256))
        layers.append(nn.ReLU())

        layers.append(norm(nn.ConvTranspose2d(256,128,kernel_size = 4,stride = 2, padding = 1)))
        layers.append(torch.nn.BatchNorm2d(128))
        layers.append(nn.ReLU())

        layers.append(norm(nn.ConvTranspose2d(128,3,kernel_size = 4,stride = 2, padding = 1)))
        layers.append(torch.nn.Tanh())
        #layers.append(torch.nn.LeakyRelu(negative_slope=0.2))    
        self.enc = nn.Sequential(*layers).to(device)                     
        print(self)
    
    def forward(self, z):
        z = z.view(-1,latent_dim,1,1)
        return self.enc(z.to(device))

In [7]:
class Data_Network_xz(nn.Module):
    def __init__(self,is_training=False, getter=None, reuse=False,
        do_spectral_norm=True):
        super(Data_Network_xz, self).__init__()
        x_layers = []
        z_layers = []
        norm = lambda x : nn.utils.spectral_norm(x) if do_spectral_norm else lambda x: x
        
        x_layers.append(norm(nn.Conv2d(3,128,kernel_size = 4, stride = 2, padding = 1)))
        x_layers.append(nn.BatchNorm2d(128))
        x_layers.append(nn.LeakyReLU(0.2))
                        
        x_layers.append(norm(nn.Conv2d(128,256,kernel_size = 4, stride = 2, padding = 1)))
        x_layers.append(nn.BatchNorm2d(256))
        x_layers.append(nn.LeakyReLU(0.2))
        
        x_layers.append(norm(nn.Conv2d(256,512,kernel_size = 4, stride = 2, padding = 1)))
        x_layers.append(nn.BatchNorm2d(512))
        x_layers.append(nn.LeakyReLU(0.2))
                                                
        z_layers.append(norm(nn.Conv2d(latent_dim,512,kernel_size=1, stride = 1)))
        z_layers.append(nn.LeakyReLU(0.2))
        z_layers.append(nn.Dropout2d(p=0.2))
                        
        z_layers.append(norm(nn.Conv2d(512,512,kernel_size=1, stride = 1)))
        z_layers.append(nn.LeakyReLU(0.2))
        z_layers.append(nn.Dropout2d(p=0.2))
                        
        self.x_net =nn.Sequential(*x_layers).to(device)                     
        self.z_net = nn.Sequential(*z_layers).to(device)                     
       
        y_layers = []
        
        y_layers.append(norm(nn.Conv2d((512*17),1024,kernel_size = 1)))
        y_layers.append(nn.LeakyReLU(0.2))
        y_layers.append(nn.Dropout2d(p=0.2))
                
        self.intermediate_net = nn.Sequential(*y_layers).to(device) 
                        
        y_layers.append(norm(nn.Conv2d(1024,1,kernel_size = 1)))
        y_layers.append(nn.LeakyReLU(0.2))
        y_layers.append(nn.Dropout2d(p=0.2))
            
        self.y_net = nn.Sequential(*y_layers).to(device) 
    
    def forward(self, x , z):
        
        x_i = self.x_net(x.to(device))
        z_i = z.view([-1, latent_dim, 1, 1])
        z_i = self.z_net(z_i.to(device))
        
        x_i = x_i.view([-1,512*4*4,1,1])
        
        y = torch.cat((x_i,z_i),1)
        res1,res2 = self.y_net(y), self.intermediate_net(y)
        del y, x_i, z_i
        # Strange behaviour of garbage collector!
        return res1,res2
     
                        
class Data_Network_xx(nn.Module):
    def __init__(self,is_training=False, getter=None, reuse=False, do_spectral_norm=True):
        super(Data_Network_xx, self).__init__()
        norm = lambda x : nn.utils.spectral_norm(x) if do_spectral_norm else lambda x: x
        layers = []
        layers.append(norm(nn.Conv2d(6,64,kernel_size = 5, stride = 2, padding = 2)))
        layers.append(nn.LeakyReLU(0.2))
        layers.append(nn.Dropout2d(p=0.2))
        layers.append(norm(nn.Conv2d(64,128,kernel_size = 5, padding = 2,stride = 2)))
        layers.append(nn.LeakyReLU(0.2))
        layers.append(nn.Dropout2d(p=0.2))
        self.net = nn.Sequential(*layers).to(device) 
        self.dense = norm(nn.Linear(128*8*8,1)).to(device)
        
    def forward(self, x , x_rec):             
        y = torch.cat((x.to(device),x_rec.to(device)),1)
        intermediate_layer = self.net(y).view(-1,128*8*8)
        logits = self.dense(intermediate_layer)
        del y
        return logits, intermediate_layer

class Data_Network_zz(nn.Module):
    def __init__(self,is_training=False, getter=None, reuse=False, do_spectral_norm=True):
        super(Data_Network_zz, self).__init__()
        norm = lambda x : nn.utils.spectral_norm(x) if do_spectral_norm else lambda x: x
        layers = []
        layers.append(norm(nn.Linear(100*2,64)))
        layers.append(nn.LeakyReLU(0.2))
        layers.append(nn.Dropout2d(p=0.2))
        layers.append(norm(nn.Linear(64,32)))
        layers.append(nn.LeakyReLU(0.2))
        layers.append(nn.Dropout2d(p=0.2))
        self.net = nn.Sequential(*layers).to(device)
        
        self.dense = norm(nn.Linear(32,1)).to(device)
        
    def forward(self, z , z_rec):
        z = z.view([-1, latent_dim])
        z_rec = z_rec.view([-1,latent_dim])
        y = torch.cat((z.to(device),z_rec.to(device)),1)
        intermediate_layer = self.net(y).view(-1,32)
        logits = self.dense(intermediate_layer)
        del z, z_rec,y
        
        return logits, intermediate_layer


In [8]:

dn_zz = Data_Network_zz()
dn_xx = Data_Network_xx()
dn_xz = Data_Network_xz()

gen = Decoder_Network()
inf = Encoder_Network()

/home/austrotaxus/anaconda3/envs/marsanomalies/lib/python3.6/site-packages/torch/cuda/__init__.py:116: UserWarning: 
    Found GPU0 GeForce GTX 660 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


Decoder_Network(
  (enc): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): Tanh()
  )
)


In [9]:
import logging
import pickle
import math
import os
import errno
import tarfile
import shutil
import numpy as np
import urllib3
from sklearn.model_selection import train_test_split
from utils.adapt_data import adapt_labels_outlier_task

logger = logging.getLogger(__name__)

def get_train(label=-1, centered=True, normalize=True):
    return _get_adapted_dataset("train", label, centered, normalize)

def get_test(label=-1, centered=True, normalize=True):
    return _get_adapted_dataset("test", label, centered, normalize)

def get_valid(label=-1, centered=True, normalize=True):
    return _get_adapted_dataset("valid", label, centered, normalize)
    
def get_shape_input():
    return (None, 32, 32, 3)

def get_shape_input_flatten():
    return (None, 32*32*3)

def get_shape_label():
    return (None,)

def num_classes():
    return 10

def get_anomalous_proportion():
    return 0.9

def _unpickle_file(filename):
    logger.debug("Loading pickle file: {}".format(filename))

    with open(filename, mode='rb') as file:
        data = pickle.load(file, encoding='bytes')

    # Reorder the data
    img = data[b'data']
    img = img.reshape([-1, 3, 32, 32])
    
    # I need channels first
    #img = img.transpose([0, 2, 3, 1])
    
    # Load labels
    lbl = np.array(data[b'labels'])

    return img, lbl

def _get_dataset(split, centered=False, normalize=False):
    '''
    Gets the adapted dataset for the experiments
    Args : 
            split (str): train or test
            normalize (bool): (Default=True) normalize data
            centered (bool): (Default=False) data centered to [-1, 1]
    Returns : 
            (tuple): <training, testing> images and labels
    '''
    path = "data"
    dirname = "cifar-10-batches-py"
    data_url = "http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

    if not os.path.exists(os.path.join(path, dirname)):
        # Extract or download data
        try:
            os.makedirs(path)
        except OSError as exception:
            if exception.errno != errno.EEXIST:
                raise
        
        file_path = os.path.join(path, data_url.split('/')[-1])
        if not os.path.exists(file_path):
            # Download
            logger.warn("Downloading {}".format(data_url))
            with urllib3.PoolManager().request('GET', data_url, preload_content=False) as r, \
                 open(file_path, 'wb') as w:
                    shutil.copyfileobj(r, w)

        logger.warn("Unpacking {}".format(file_path))
        # Unpack data
        tarfile.open(name=file_path, mode="r:gz").extractall(path)

    # Import the data
    if split == 'test':
        filenames = ["test_batch"] 
    # elif split == 'valid':
    #     filenames = ["data_batch_5"]
    else:
        filenames = ["data_batch_{}".format(i) for i in range(1, 6)]
    
    imgs = []
    lbls = []
    for f in filenames:
        img, lbl = _unpickle_file(os.path.join(path, dirname, f))
        imgs.append(img)
        lbls.append(lbl)

    # Now we flatten the arrays
    imgs = np.concatenate(imgs)
    lbls = np.concatenate(lbls)

    # Convert images to [0..1] range
    if normalize:
        imgs = imgs.astype(np.float32)/255.0
    if centered:
        imgs = imgs.astype(np.float32)*2. - 1.
    return imgs.astype(np.float32), lbls

def _get_adapted_dataset(split, label=None, centered=False, normalize=False):
    """
    Gets the adapted dataset for the experiments
    Args : 
            split (str): train or test
            mode (str): inlier or outlier
            label (int): int in range 0 to 10, is the class/digit
                         which is considered inlier or outlier
            rho (float): proportion of anomalous classes INLIER
                         MODE ONLY
            centered (bool): (Default=False) data centered to [-1, 1]
    Returns : 
            (tuple): <training, testing> images and labels
    """
    dataset = {}
    dataset['x_train'],  dataset['y_train'] = _get_dataset('train',
                                                           centered=centered,
                                                           normalize=normalize)
    dataset['x_test'], dataset['y_test'] = _get_dataset('test', centered=centered,
                                                           normalize=normalize)

    full_x_data = np.concatenate([dataset['x_train'], dataset['x_test']], axis=0)
    full_y_data = np.concatenate([dataset['y_train'], dataset['y_test']], axis=0)
    
    full_y_data[full_y_data == 10] = 0

    dataset['x_train'], dataset['x_test'], \
    dataset['y_train'], dataset['y_test'] = train_test_split(full_x_data,
                                                             full_y_data,
                                                             test_size=0.2,
                                                             random_state=42)
    
    dataset['x_train'], dataset['x_valid'], \
    dataset['y_train'], dataset['y_valid'] = train_test_split(dataset['x_train'],
                                                             dataset['y_train'],
                                                             test_size=0.25,
                                                             random_state=42)


    key_img = 'x_' + split
    key_lbl = 'y_' + split

    if label != -1:

        if split in ['train', 'valid']:

            inliers = dataset[key_img][dataset[key_lbl] == label], \
                      dataset[key_lbl][dataset[key_lbl] == label]
            outliers = dataset[key_img][dataset[key_lbl] != label], \
                       dataset[key_lbl][dataset[key_lbl] != label]

            dataset[key_img], dataset[key_lbl] = inliers

            dataset[key_lbl] = adapt_labels_outlier_task(dataset[key_lbl],
                                                         label)
            return (dataset[key_img], dataset[key_lbl])
        else:
            dataset[key_lbl] = adapt_labels_outlier_task(dataset[key_lbl],
                                                         label)

            return (dataset[key_img], dataset[key_lbl])

In [10]:
random_seed = 42
rng = np.random.RandomState(random_seed)

trainx, trainy =  _get_dataset("train")
testx, testy = _get_dataset("test")
nr_batches_train = int(trainx.shape[0] / batch_size)
nr_batches_test = int(testx.shape[0] / batch_size)

In [11]:
import subprocess

def get_gpu_memory_map():
    """Get the current gpu usage.

    Returns
    -------
    usage: dict
        Keys are device ids as integers.
        Values are memory usage as integers in MB.
    """
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ], encoding='utf-8')
    # Convert lines into a dictionary
    gpu_memory = [int(x) for x in result.strip().split('\n')]
    gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
    return gpu_memory_map

In [12]:
disc_params = list(dn_xz.parameters())+list(dn_xx.parameters()) +list(dn_zz.parameters())
disc_optimizer = torch.optim.Adam( disc_params, lr=learning_rate,betas = (0.5,0.999))
gen_params = list(gen.parameters()) + list(inf.parameters())
gen_optimizer = torch.optim.Adam(gen_params , lr = learning_rate, betas = (0.5,0.999))

In [13]:
# With every batch GPU usage increases 
# Didnt found rational reasons of such behaviour
def train(trainx):
    import gc
    epoch = 0
    nb_epochs = 100
    trainx = trainx[rng.permutation(trainx.shape[0])]
    FG, FD = [],[]
    i = 0
    
    bce = nn.BCELoss()
    sigmoid = nn.Sigmoid()
    while not False and epoch < nb_epochs:
    
        # construct randomly permuted minibatches
        # shuffling dataset
 
        train_loss_dis_xz, train_loss_dis_xx,  train_loss_dis_zz, \
        train_loss_dis, train_loss_gen, train_loss_enc = [0, 0, 0, 0, 0, 0]
    
            # Training
        for t in range(nr_batches_train):
            display_progression_epoch(t, nr_batches_train)
            
           
       
            ran_from = t * batch_size
            ran_to = (t + 1) * batch_size
        
           
            x = torch.Tensor(trainx[ran_from:ran_to])
            z = torch.Tensor(np.random.normal(size = [batch_size, latent_dim]))
            p_x = gen(z)
            q_z = inf(x)
        
          
            decoder_logit, _ = dn_xz(p_x, z)
            encoder_logit, _ = dn_xz(x , q_z)
    
          
            decoder_loss = bce(sigmoid(decoder_logit),torch.zeros_like(decoder_logit))
            encoder_loss =  bce(sigmoid(encoder_logit),torch.ones_like(encoder_logit))
            dis_loss_xz = torch.mean(encoder_loss+decoder_loss)
            decoder_loss2 =  bce(sigmoid(decoder_logit),torch.ones_like(decoder_logit))
            encoder_loss2 = bce(sigmoid(encoder_logit),torch.zeros_like(encoder_logit))
          
            gen_loss_xz = torch.mean((decoder_loss2))  + (torch.mean(encoder_loss2))
          
            rec_z = inf(p_x)
            rec_x = gen(q_z)
           
            x_logit_real,_ = dn_xx(x,x)
            x_logit_fake,_ = dn_xx(x,rec_x)
            z_logit_real,_ = dn_zz(z,z)
            z_logit_fake,_ = dn_zz(z,rec_z)
        
            x_sigmoid_real = bce(sigmoid(x_logit_real),torch.ones_like(x_logit_real))
            x_sigmoid_fake = bce(sigmoid(x_logit_fake),torch.zeros_like(x_logit_fake))
            x_sigmoid_real2 = bce(sigmoid(x_logit_real),torch.zeros_like(x_logit_real))
            x_sigmoid_fake2 = bce(sigmoid(x_logit_fake),torch.ones_like(x_logit_fake))
            z_sigmoid_real = bce(sigmoid(z_logit_real),torch.ones_like(z_logit_real))
            z_sigmoid_fake = bce(sigmoid(z_logit_fake),torch.zeros_like(z_logit_fake))
            z_sigmoid_real2 = bce(sigmoid(z_logit_real),torch.zeros_like(z_logit_real))
            z_sigmoid_fake2 = bce(sigmoid(z_logit_fake),torch.ones_like(z_logit_fake))
          
            dis_loss_x = torch.mean(x_sigmoid_real + x_sigmoid_fake)
            dis_loss_z = torch.mean(z_sigmoid_real + z_sigmoid_fake)
            disc_loss = dis_loss_xz + dis_loss_x + dis_loss_z
          
            cost_x = torch.mean(x_sigmoid_real2 + x_sigmoid_fake2) 
            cost_z = torch.mean(z_sigmoid_real2 + z_sigmoid_fake2)
            gen_loss = gen_loss_xz + cost_x  + cost_z
    
    
    
            gen_optimizer.zero_grad()
            gen_loss.backward(retain_graph = True)
            gen_optimizer.step()
            f_g = [gen_loss.data, gen_loss_xz.data, cost_x.data ,cost_z.data]
            
            disc_optimizer.zero_grad()
            disc_loss.backward(retain_graph = False)
            disc_optimizer.step()
        
            disc_loss.detach()
            gen_loss.detach()
            torch.cuda.empty_cache()
            gc.collect()
            f_d = disc_loss.data
            
            FG.append(f_g)
            FD.append(f_d)
            i+=1
            print("Happy New %d Batch!"%i)
            print("Memory map: " + str(get_gpu_memory_map()))
    
        print("epoch %d iter %d: discloss %f genloss %f adv_x %f recons_x %f " % (epoch, i, f_d, f_g[0], f_g[1], f_g[2]))
    
train(trainx)

Happy New 1 Batch!
Memory map: {0: 1237}
Happy New 2 Batch!
Memory map: {0: 1239}
Happy New 3 Batch!
Memory map: {0: 1205}
Happy New 4 Batch!
Memory map: {0: 1206}
Happy New 5 Batch!
Memory map: {0: 1205}
Happy New 6 Batch!
Memory map: {0: 1205}
Happy New 7 Batch!
Memory map: {0: 1204}
Happy New 8 Batch!
Memory map: {0: 1205}
Happy New 9 Batch!
Memory map: {0: 1205}
Happy New 10 Batch!
Memory map: {0: 1206}
Happy New 11 Batch!
Memory map: {0: 1206}
Happy New 12 Batch!
Memory map: {0: 1205}
Happy New 13 Batch!
Memory map: {0: 1206}
Happy New 14 Batch!
Memory map: {0: 1206}
Happy New 15 Batch!
Memory map: {0: 1206}
Happy New 16 Batch!
Memory map: {0: 1205}
Happy New 17 Batch!
Memory map: {0: 1206}
Happy New 18 Batch!
Memory map: {0: 1206}
Happy New 19 Batch!
Memory map: {0: 1206}
Happy New 20 Batch!
Memory map: {0: 1207}
Happy New 21 Batch!
Memory map: {0: 1206}
Happy New 22 Batch!
Memory map: {0: 1205}
Happy New 23 Batch!
Memory map: {0: 1205}
Happy New 24 Batch!
Memory map: {0: 1205}
H

Happy New 194 Batch!
Memory map: {0: 1202}
Happy New 195 Batch!
Memory map: {0: 1203}
Happy New 196 Batch!
Memory map: {0: 1203}
Happy New 197 Batch!
Memory map: {0: 1203}
Happy New 198 Batch!
Memory map: {0: 1203}
Happy New 199 Batch!
Memory map: {0: 1203}
Happy New 200 Batch!
Memory map: {0: 1203}
Happy New 201 Batch!
Memory map: {0: 1203}
Happy New 202 Batch!
Memory map: {0: 1202}
Happy New 203 Batch!
Memory map: {0: 1203}
Happy New 204 Batch!
Memory map: {0: 1203}
Happy New 205 Batch!
Memory map: {0: 1200}
Happy New 206 Batch!
Memory map: {0: 1201}
Happy New 207 Batch!
Memory map: {0: 1201}
Happy New 208 Batch!
Memory map: {0: 1202}
Happy New 209 Batch!
Memory map: {0: 1202}
Happy New 210 Batch!
Memory map: {0: 1201}
Happy New 211 Batch!
Memory map: {0: 1202}
Happy New 212 Batch!
Memory map: {0: 1203}
Happy New 213 Batch!
Memory map: {0: 1202}
Happy New 214 Batch!
Memory map: {0: 1202}
Happy New 215 Batch!
Memory map: {0: 1202}
Happy New 216 Batch!
Memory map: {0: 1202}
Happy New 2

Happy New 385 Batch!
Memory map: {0: 1204}
Happy New 386 Batch!
Memory map: {0: 1204}
Happy New 387 Batch!
Memory map: {0: 1205}
Happy New 388 Batch!
Memory map: {0: 1204}
Happy New 389 Batch!
Memory map: {0: 1204}
Happy New 390 Batch!
Memory map: {0: 1204}
Happy New 391 Batch!
Memory map: {0: 1204}
Happy New 392 Batch!
Memory map: {0: 1204}
Happy New 393 Batch!
Memory map: {0: 1204}
Happy New 394 Batch!
Memory map: {0: 1205}
Happy New 395 Batch!
Memory map: {0: 1206}
Happy New 396 Batch!
Memory map: {0: 1205}
Happy New 397 Batch!
Memory map: {0: 1204}
Happy New 398 Batch!
Memory map: {0: 1204}
Happy New 399 Batch!
Memory map: {0: 1204}
Happy New 400 Batch!
Memory map: {0: 1205}
Happy New 401 Batch!
Memory map: {0: 1205}
Happy New 402 Batch!
Memory map: {0: 1205}
Happy New 403 Batch!
Memory map: {0: 1204}
Happy New 404 Batch!
Memory map: {0: 1204}
Happy New 405 Batch!
Memory map: {0: 1204}
Happy New 406 Batch!
Memory map: {0: 1205}
Happy New 407 Batch!
Memory map: {0: 1204}
Happy New 4

Happy New 576 Batch!
Memory map: {0: 1202}
Happy New 577 Batch!
Memory map: {0: 1201}
Happy New 578 Batch!
Memory map: {0: 1202}
Happy New 579 Batch!
Memory map: {0: 1202}
Happy New 580 Batch!
Memory map: {0: 1202}
Happy New 581 Batch!
Memory map: {0: 1202}
Happy New 582 Batch!
Memory map: {0: 1200}
Happy New 583 Batch!
Memory map: {0: 1200}
Happy New 584 Batch!
Memory map: {0: 1200}
Happy New 585 Batch!
Memory map: {0: 1199}
Happy New 586 Batch!
Memory map: {0: 1200}
Happy New 587 Batch!
Memory map: {0: 1201}
Happy New 588 Batch!
Memory map: {0: 1201}
Happy New 589 Batch!
Memory map: {0: 1201}
Happy New 590 Batch!
Memory map: {0: 1200}
Happy New 591 Batch!
Memory map: {0: 1200}
Happy New 592 Batch!
Memory map: {0: 1200}
Happy New 593 Batch!
Memory map: {0: 1201}
Happy New 594 Batch!
Memory map: {0: 1201}
Happy New 595 Batch!
Memory map: {0: 1200}
Happy New 596 Batch!
Memory map: {0: 1200}
Happy New 597 Batch!
Memory map: {0: 1201}
Happy New 598 Batch!
Memory map: {0: 1202}
Happy New 5

Happy New 767 Batch!
Memory map: {0: 1200}
Happy New 768 Batch!
Memory map: {0: 1200}
Happy New 769 Batch!
Memory map: {0: 1200}
Happy New 770 Batch!
Memory map: {0: 1200}
Happy New 771 Batch!
Memory map: {0: 1200}
Happy New 772 Batch!
Memory map: {0: 1201}
Happy New 773 Batch!
Memory map: {0: 1205}
Happy New 774 Batch!
Memory map: {0: 1203}
Happy New 775 Batch!
Memory map: {0: 1205}
Happy New 776 Batch!
Memory map: {0: 1204}
Happy New 777 Batch!
Memory map: {0: 1204}
Happy New 778 Batch!
Memory map: {0: 1205}
Happy New 779 Batch!
Memory map: {0: 1205}
Happy New 780 Batch!
Memory map: {0: 1205}
Happy New 781 Batch!
Memory map: {0: 1206}
Happy New 782 Batch!
Memory map: {0: 1205}
Happy New 783 Batch!
Memory map: {0: 1205}
Happy New 784 Batch!
Memory map: {0: 1204}
Happy New 785 Batch!
Memory map: {0: 1204}
Happy New 786 Batch!
Memory map: {0: 1204}
Happy New 787 Batch!
Memory map: {0: 1204}
Happy New 788 Batch!
Memory map: {0: 1204}
Happy New 789 Batch!
Memory map: {0: 1204}
Happy New 7

Happy New 958 Batch!
Memory map: {0: 1203}
Happy New 959 Batch!
Memory map: {0: 1203}
Happy New 960 Batch!
Memory map: {0: 1203}
Happy New 961 Batch!
Memory map: {0: 1203}
Happy New 962 Batch!
Memory map: {0: 1203}
Happy New 963 Batch!
Memory map: {0: 1203}
Happy New 964 Batch!
Memory map: {0: 1203}
Happy New 965 Batch!
Memory map: {0: 1203}
Happy New 966 Batch!
Memory map: {0: 1203}
Happy New 967 Batch!
Memory map: {0: 1203}
Happy New 968 Batch!
Memory map: {0: 1203}
Happy New 969 Batch!
Memory map: {0: 1203}
Happy New 970 Batch!
Memory map: {0: 1203}
Happy New 971 Batch!
Memory map: {0: 1203}
Happy New 972 Batch!
Memory map: {0: 1203}
Happy New 973 Batch!
Memory map: {0: 1203}
Happy New 974 Batch!
Memory map: {0: 1203}
Happy New 975 Batch!
Memory map: {0: 1202}
Happy New 976 Batch!
Memory map: {0: 1202}
Happy New 977 Batch!
Memory map: {0: 1202}
Happy New 978 Batch!
Memory map: {0: 1202}
Happy New 979 Batch!
Memory map: {0: 1203}
Happy New 980 Batch!
Memory map: {0: 1204}
Happy New 9

Happy New 1145 Batch!
Memory map: {0: 1200}
Happy New 1146 Batch!
Memory map: {0: 1201}
Happy New 1147 Batch!
Memory map: {0: 1202}
Happy New 1148 Batch!
Memory map: {0: 1200}
Happy New 1149 Batch!
Memory map: {0: 1200}
Happy New 1150 Batch!
Memory map: {0: 1200}
Happy New 1151 Batch!
Memory map: {0: 1200}
Happy New 1152 Batch!
Memory map: {0: 1200}
Happy New 1153 Batch!
Memory map: {0: 1199}
Happy New 1154 Batch!
Memory map: {0: 1200}
Happy New 1155 Batch!
Memory map: {0: 1201}
Happy New 1156 Batch!
Memory map: {0: 1201}
Happy New 1157 Batch!
Memory map: {0: 1201}
Happy New 1158 Batch!
Memory map: {0: 1200}
Happy New 1159 Batch!
Memory map: {0: 1200}
Happy New 1160 Batch!
Memory map: {0: 1199}
Happy New 1161 Batch!
Memory map: {0: 1200}
Happy New 1162 Batch!
Memory map: {0: 1201}
Happy New 1163 Batch!
Memory map: {0: 1202}
Happy New 1164 Batch!
Memory map: {0: 1200}
Happy New 1165 Batch!
Memory map: {0: 1200}
Happy New 1166 Batch!
Memory map: {0: 1200}
Happy New 1167 Batch!
Memory map

Happy New 1331 Batch!
Memory map: {0: 1202}
Happy New 1332 Batch!
Memory map: {0: 1201}
Happy New 1333 Batch!
Memory map: {0: 1201}
Happy New 1334 Batch!
Memory map: {0: 1201}
Happy New 1335 Batch!
Memory map: {0: 1201}
Happy New 1336 Batch!
Memory map: {0: 1201}
Happy New 1337 Batch!
Memory map: {0: 1201}
Happy New 1338 Batch!
Memory map: {0: 1202}
Happy New 1339 Batch!
Memory map: {0: 1202}
Happy New 1340 Batch!
Memory map: {0: 1202}
Happy New 1341 Batch!
Memory map: {0: 1201}
Happy New 1342 Batch!
Memory map: {0: 1201}
Happy New 1343 Batch!
Memory map: {0: 1202}
Happy New 1344 Batch!
Memory map: {0: 1202}
Happy New 1345 Batch!
Memory map: {0: 1201}
Happy New 1346 Batch!
Memory map: {0: 1201}
Happy New 1347 Batch!
Memory map: {0: 1201}
Happy New 1348 Batch!
Memory map: {0: 1201}
Happy New 1349 Batch!
Memory map: {0: 1202}
Happy New 1350 Batch!
Memory map: {0: 1202}
Happy New 1351 Batch!
Memory map: {0: 1203}
Happy New 1352 Batch!
Memory map: {0: 1203}
Happy New 1353 Batch!
Memory map

Happy New 1517 Batch!
Memory map: {0: 1200}
Happy New 1518 Batch!
Memory map: {0: 1200}
Happy New 1519 Batch!
Memory map: {0: 1201}
Happy New 1520 Batch!
Memory map: {0: 1202}
Happy New 1521 Batch!
Memory map: {0: 1201}
Happy New 1522 Batch!
Memory map: {0: 1202}
Happy New 1523 Batch!
Memory map: {0: 1202}
Happy New 1524 Batch!
Memory map: {0: 1201}
Happy New 1525 Batch!
Memory map: {0: 1202}
Happy New 1526 Batch!
Memory map: {0: 1201}
Happy New 1527 Batch!
Memory map: {0: 1200}
Happy New 1528 Batch!
Memory map: {0: 1200}
Happy New 1529 Batch!
Memory map: {0: 1200}
Happy New 1530 Batch!
Memory map: {0: 1201}
Happy New 1531 Batch!
Memory map: {0: 1202}
Happy New 1532 Batch!
Memory map: {0: 1202}
Happy New 1533 Batch!
Memory map: {0: 1202}
Happy New 1534 Batch!
Memory map: {0: 1202}
Happy New 1535 Batch!
Memory map: {0: 1201}
Happy New 1536 Batch!
Memory map: {0: 1200}
Happy New 1537 Batch!
Memory map: {0: 1200}
Happy New 1538 Batch!
Memory map: {0: 1200}
Happy New 1539 Batch!
Memory map

NameError: name 'f_d' is not defined

## Memory leakage
For some reasons intermediate tensors calculated in dn_xz are saved in GPU memory and not removed by GC!
For example, <class 'torch.Tensor'> torch.Size([32, 8704, 1, 1]) which most probably corresponds to "y" tensor in "dn_xz.forward()" definition

In [ ]:
import gc
for obj in gc.get_objects():
    try:
        print(type(obj), obj.size())
    except:
        pass

In [17]:
"""keys = globals().keys()
for k in keys:
    print(k)
   """ 

'keys = globals().keys()\nfor k in keys:\n    print(k)\n   '

In [18]:
#del gen_loss, disc_loss, 
print(get_gpu_memory_map())


del gen_loss, disc_loss
torch.cuda.empty_cache()
gc.collect()


print(get_gpu_memory_map())

{0: 1904}


NameError: name 'gen_loss' is not defined

In [23]:
#del gen_loss, disc_loss, 
print(get_gpu_memory_map())


del dn_xx
torch.cuda.empty_cache()
gc.collect()


print(get_gpu_memory_map())

{0: 1879}
{0: 1879}


In [26]:
#del gen_loss, disc_loss, 
print(get_gpu_memory_map())


del gen
torch.cuda.empty_cache()
gc.collect()


print(get_gpu_memory_map())

{0: 1875}
{0: 1874}
